# Ejercicio 1

Escribe un programa en Python que use Biopython para hacer una búsqueda
BLAST de una secuencia de ADN que introduzcas por teclado. El programa debe mostrar por
pantalla el número de resultados obtenidos, el E-value del mejor resultado y la descripción de
la secuencia más similar.

In [2]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO
import re

#Secuencias cortas (alrededor de 20-30 nucleótidos) podrían arrojar resultados, 
#especialmente si contienen regiones o motivos altamente conservados.
#Secuencias más largas (de varios cientos de nucleótidos o más) suelen proporcionar 
#resultados más significativos y confiables, ya que tienen más información para la comparación.

# Función para validar la secuencia de proteínas
def valida_sequenza(sequenza):
    # Patrón para verificar la secuencia de proteínas
    patron = re.compile(r'^[ACGT]+$')
    return bool(patron.match(sequenza))

# Obtener la secuencia de proteína del usuario con validación
while True:
    sequence = input("Introduce la secuencia de ADN (Secuencias de mas de 20-30 nucleótidos suelen proporcionar resultados más significativos): ")
    if valida_sequenza(sequence):
        break
    else:
        print("\n La secuencia de ADN ingresada no es válida. Ingresa solo letras correspondientes a los aminoácidos (ACGT).")

print('Sequence: ',sequence)

# Realizar la búsqueda BLAST en la base de datos de nucleótidos de NCBI
result_handle = NCBIWWW.qblast("blastn", "nt", sequence)


blast_records = NCBIXML.parse(result_handle)

num_results = 0
best_e_value = float('inf')
best_hit_description = ''

for record in blast_records:
    for alignment in record.alignments:
        num_results += 1
        for hsp in alignment.hsps:
            if hsp.expect < best_e_value:
                best_e_value = hsp.expect
                best_hit_description = alignment.title

print("Número de resultados obtenidos:", num_results)
print("El mejor E-value obtenido:", best_e_value)
print("Descripción de la secuencia más similar:", best_hit_description)


Introduce la secuencia de ADN (Secuencias de 20-30 nucleótidos suelen proporcionar resultados más significativos):  ACTGCTAVA


La secuencia de proteína ingresada no es válida. Ingresa solo letras correspondientes a los aminoácidos (ACDEFGHIKLMNPQRSTVWY).


Introduce la secuencia de ADN (Secuencias de 20-30 nucleótidos suelen proporcionar resultados más significativos):  ACGTACGATACGATACAGCTAGACATAGCATGACA


Sequence:  ACGTACGATACGATACAGCTAGACATAGCATGACA
Número de resultados obtenidos: 7
El mejor E-value obtenido: 1.99993
Descripción de la secuencia más similar: gi|2306605176|emb|OX244029.2| Tridacna gigas genome assembly, chromosome: 2


# Ejercicio 2

Escribe un programa en Python que use Biopython para hacer una búsqueda
BLAST de una secuencia de proteína que introduzcas por teclado. El programa debe guardar
en un fichero los resultados que tengan un E-value menor que 0.001. El fichero debe contener
el identificador, la longitud, el E-value y el porcentaje de identidad de cada resultado.

In [5]:
from Bio.Blast import NCBIWWW, NCBIXML
import re

#proteina prueba
#MAEGEITTFTALTEKFNLPPGNYKKPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHIQLQLSAESVGEVYIKSTETGQYLAMDTSGLLYGSQTPSEECLFLERRLEENHYNTYTSKKHAEKNWFVGLKKNGSCKRGPRTHYGQKAILFLPLPV


# Función para validar la secuencia de proteínas
def valida_sequenza_proteina(sequenza):
    # Patrón para verificar la secuencia de proteínas
    patron = re.compile(r'^[ACDEFGHIKLMNPQRSTVWYacdefghiklmnpqrstvwy]+$')
    return bool(patron.match(sequenza))

# Obtener la secuencia de proteína del usuario con validación
while True:
    protein_sequence = input("Introduce la secuencia de proteína (Secuencias de mas de 100 aminoácidos suelen proporcionar resultados más significativos): ")
    if valida_sequenza_proteina(protein_sequence):
        break
    else:
        print("La secuencia de proteína ingresada no es válida. Ingresa solo letras correspondientes a los aminoácidos (ACDEFGHIKLMNPQRSTVWY).")

# Realizar la búsqueda BLAST en la base de datos de proteínas de NCBI
result_handle = NCBIWWW.qblast("blastp", "nr", protein_sequence)

# Analizar los resultados obtenidos
blast_records = NCBIXML.parse(result_handle)

# Crear un archivo para guardar los resultados con E-value < 0.001
with open("resultados_blast.txt", "w") as file_output:
    file_output.write("Identificador\tLongitud\tE-value\t% Identidad\n")
    
    for record in blast_records:
        for alignment in record.alignments:
            for hsp in alignment.hsps:
                # Guardar resultados con E-value < 0.001 en el archivo
                if hsp.expect < 0.001:
                    identificador = alignment.title.split()[1]  # Obtener el identificador del título
                    longitud = alignment.length
                    e_value = hsp.expect
                    porcentaje_identidad = hsp.identities / hsp.align_length * 100
                    
                    # Escribir los datos en el archivo
                    file_output.write(f"{identificador}\t{longitud}\t{e_value}\t{porcentaje_identidad:.2f}%\n")

print("Los resultados con E-value < 0.001 han sido guardados en 'resultados_blast.txt'.")


Introduce la secuencia de proteína (Secuencias de mas de 20-30 aminoácidos suelen proporcionar resultados más significativos):  MAEGEITTFTALTEKFNLPPGNYKKPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHIQLQLSAESVGEVYIKSTETGQYLAMDTSGLLYGSQTPSEECLFLERRLEENHYNTYTSKKHAEKNWFVGLKKNGSCKRGPRTHYGQKAILFLPLPV


Los resultados con E-value < 0.001 han sido guardados en 'resultados_blast.txt'.


# Ejercicio 3

Escribe un programa en Python que use Biopython para hacer una búsqueda
BLAST de una secuencia de ADN que leas de un fichero en formato FASTA. El programa
debe hacer la búsqueda en la base de datos nr y filtrar los resultados por el organismo que
elijas. El programa debe mostrar por pantalla el número de resultados obtenidos y el E-value
medio de los mismos.

In [1]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO

def filter_by_organism(results, organism):
    filtered_results = []
    for alignment in results:
        for desc in alignment.descriptions:
            if organism.lower() in desc.title.lower():
                filtered_results.append(alignment)
                break  # Move to the next result once the organism is found
    return filtered_results

fasta_file = "secuencia_adn.fasta"
dna_sequence = SeqIO.read(fasta_file, "fasta")

blast_results = NCBIWWW.qblast("blastn", "nr", dna_sequence.seq)
parsed_results = NCBIXML.parse(blast_results)

organism_to_filter = "Homo sapiens"  # Modify this to your preferred organism

filtered_results = filter_by_organism(parsed_results, organism_to_filter)

total_e_value = 0.0
num_hsps = 0

for alignment in filtered_results:
    for hsp in alignment.alignments:
        for h in hsp.hsps:
            # Process each HSP (High-scoring Segment Pair)
            total_e_value += h.expect
            num_hsps += 1

if num_hsps > 0:
    average_e_value = total_e_value / num_hsps
else:
    average_e_value = 0.0

print(f"Number of results obtained for {organism_to_filter}: {len(filtered_results)}")
print(f"Average E-value of the results: {average_e_value}")



Number of results obtained for Homo sapiens: 1
Average E-value of the results: 0.0
